# Mining testimonial fragments of the Holocaust

**Experience domain:** numbness

### Load the necessary libraries

In [1]:
import sys; sys.path.insert(0, '..')
import itertools

In [2]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [3]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [4]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [5]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [6]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [7]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [8]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [9]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [10]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [11]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [12]:
main_node = "numbness"
#delete_main_node("numbness")
add_main_node('numbness')

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

In [13]:
query = '[lemma="numb"]'

result = topic_concordancer.main(query,window=50,topicn=25)

### Print the key topics

for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

### Analyze documents

i=0
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

## Testimonial fragments

### 1.  Kill

In [14]:
lemmas = ['kill','numb']

In [15]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="kill"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="kill"])


In [16]:
domain_term = 'kill'

In [17]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [18]:
fragment_1 = {}
fragment_1['original_sentence'] = "[PAUSES FOR 3 SECONDS] Then they said, you're out of luck because we're going to kill you all. [PAUSES FOR 4 SECONDS] We [PAUSES FOR 4 SECONDS] were numb."
fragment_1['label']="Then they said, you're out of luck because we're going to kill you all. We were numb."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%223%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22Then%22%5D+%5B%22they%22%5D+%5B%22said%22%5D+%5B%5D+%5B%22you%22%5D+%5B%5D+%5B%22out%22%5D+%5B%22of%22%5D+%5B%22luck%22%5D+%5B%22because%22%5D+%5B%22we%22%5D+%5B%5D+%5B%22going%22%5D+%5B%22to%22%5D+%5B%22kill%22%5D+%5B%22you%22%5D+%5B%22all%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%224%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22We%22%5D+%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%224%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22were%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "men . [ PAUSES FOR 3 SECONDS ] Then they said , you 're out of luck because we 're going to kill you all . [ PAUSES FOR 4 SECONDS ] We [ PAUSES FOR 4 SECONDS ] were numb . ", 'right': '', 'complete_match': "men . [ 

In [19]:
fragment_2 = {}
fragment_2['original_sentence'] = "To see another 10,000 Jews come and be killed, another 20,000 be killed? That was not...that we wanted to see. You know. We had to. Then after awhile you become so numb that you just...you don't react, you...you know?"
fragment_2['label']="To see another 10,000 Jews come and be killed (..) Then after awhile you become so numb (..)"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22To%22%5D+%5B%22see%22%5D+%5B%22another%22%5D+%5B%5D+%5B%22Jews%22%5D+%5B%22come%22%5D+%5B%22and%22%5D+%5B%22be%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22another%22%5D+%5B%5D+%5B%22be%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22That%22%5D+%5B%22was%22%5D+%5B%22not%22%5D+%5B%5D%7B0%2C50%7D+%5B%22that%22%5D+%5B%22we%22%5D+%5B%22wanted%22%5D+%5B%22to%22%5D+%5B%22see%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%5D+%5B%22Then%22%5D+%5B%22after%22%5D+%5B%22awhile%22%5D+%5B%22you%22%5D+%5B%22become%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%22that%22%5D+%5B%22you%22%5D+%5B%22just%22%5D+%5B%5D%7B0%2C50%7D+%5B%22you%22%5D+%5B%22do%22%5D+%5B%5D+%5B%22react%22%5D+%5B%5D+%5B%22you%22%5D+%5B%5D%7B0%2C50%7D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "To see another 10,000 Jews come and be killed , ano

In [20]:
fragment_3 = {}
fragment_3['original_sentence'] = 'And we were so numb, [? Cathy ?] every 10 people get killed.'
fragment_3['label']="And we were so numb, (..) every 10 people get killed."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%5D+%5B%22Cathy%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22every%22%5D+%5B%2210%22%5D+%5B%22people%22%5D+%5B%22get%22%5D+%5B%22killed%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And we were so numb , [ ? Cathy ? ] every 10 people get killed . ', 'right': '', 'complete_match': 'And we were so numb , [ ? Cathy ? ] every 10 people get killed . ', 'testimony_id': 'usc_shoah_25287', 'shelfmark': ['USC 25287'], 'token_start': 16040, 'token_end': 16057}


In [21]:
fragment_4 = {}
fragment_4['original_sentence'] = "from people being wounded, from people being killed, from cries. And finally, everything subsided. But I was alive. I didn't -- I didn't even think. I was numb."
fragment_4['label']= "I heard screams and-- from-- from-- from-- from people being wounded, from people being killed, from cries (..) I was numb"
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22from%22%5D+%5B%22people%22%5D+%5B%22being%22%5D+%5B%22wounded%22%5D+%5B%5D+%5B%22from%22%5D+%5B%22people%22%5D+%5B%22being%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22from%22%5D+%5B%22cries%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22finally%22%5D+%5B%5D+%5B%22everything%22%5D+%5B%22subsided%22%5D+%5B%5D+%5B%22But%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22alive%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22even%22%5D+%5B%22think%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "from people being wounded , from people being killed , from cries . And finally , everything subsided . But I was alive . I did n't -- I did n't even think . I was numb . ", 'right': '', 'complete_match': "from people being wounded , from people being killed , from cries . And finally , ev

In [22]:
add_testimonial_fragments(fragments)

### 2.  Shock

In [23]:
domain_term = 'shock'

In [24]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [25]:
query = '[lemma="shock"]([]{0,25}[lemma="numb"])|[lemma="numb"]([]{0,25}[lemma="shock"])'

In [26]:
domain_term = 'shock'

In [27]:
fragment_1 = {}
fragment_1['original_sentence'] = 'I was in complete shock. I was numb, nothing.'
fragment_1['label']='I was in complete shock. I was numb, nothing.'
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22was%22%5D+%5B%22in%22%5D+%5B%22complete%22%5D+%5B%22shock%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%22nothing%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I was in complete shock . I was numb , nothing . ', 'right': '', 'complete_match': 'I was in complete shock . I was numb , nothing . ', 'testimony_id': 'HVT-108', 'shelfmark': ['Fortunoff HVT-108'], 'token_start': 4794, 'token_end': 4806}


In [28]:
fragment_2 = {}
fragment_2['original_sentence'] = 'We were so numb, we were almost, the shock hypnotized us,'
fragment_2['label']='We were so numb, we were almost, the shock hypnotized us (..)'
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22were%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22almost%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22shock%22%5D+%5B%22hypnotized%22%5D+%5B%22us%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We were so numb , we were almost , the shock hypnotized us , ', 'right': '', 'complete_match': 'We were so numb , we were almost , the shock hypnotized us , ', 'testimony_id': 'irn504925', 'shelfmark': ['USHMM RG-50.549.01*0024'], 'token_start': 5647, 'token_end': 5661}


In [29]:
fragment_3 = {}
fragment_3['original_sentence'] = "You are numb. You're shocked."
fragment_3['label']="You are numb. You're shocked"
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22are%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%22You%22%5D+%5B%5D+%5B%22shocked%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "You are numb . You 're shocked . ", 'right': '', 'complete_match': "You are numb . You 're shocked . ", 'testimony_id': 'usc_shoah_10358', 'shelfmark': ['USC 10358'], 'token_start': 3799, 'token_end': 3807}


In [30]:
fragment_ = {}
fragment_4['original_sentence'] = "And [PAUSES FOR 4 SECONDS] I was numb when I was there.  I'll be very honest with you. I was not-- I was somewhat in a state of shock."
fragment_4['label']="I was numb when I was there. (..) I was somewhat in a state of shock."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%224%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22numb%22%5D+%5B%22when%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22there%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22%27ll%22%5D+%5B%22be%22%5D+%5B%22very%22%5D+%5B%22honest%22%5D+%5B%22with%22%5D+%5B%22you%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22not%22%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22somewhat%22%5D+%5B%22in%22%5D+%5B%22a%22%5D+%5B%22state%22%5D+%5B%22of%22%5D+%5B%22shock%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And [ PAUSES FOR 4 SECONDS ] I was numb when I was there . I 'll be very honest with you . I was not -- I was somewhat in a state of shock . ", 'right': '', 'complete_match': "And [ PAUSES FOR 4 SECONDS ] I was numb when I was there . I 'll be very honest with you . I was not -- I

In [31]:
fragment_5 = {}
fragment_5['original_sentence'] = 'I was 15. I was shocked.'
fragment_5['label']='I was a completely-- I guess I was-- I was 15. I was shocked. I was, uh, numb.'
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22was%22%5D+%5B%2215%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22shocked%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I was 15 . I was shocked . ', 'right': '', 'complete_match': 'I was 15 . I was shocked . ', 'testimony_id': 'usc_shoah_24470', 'shelfmark': ['USC 24470'], 'token_start': 7698, 'token_end': 7706}


In [32]:
add_testimonial_fragments(fragments)

### 3.  Dead inside

In [33]:
main_node = "numbness"
domain_term = 'shock'

In [34]:
lemmas = ['lose','numb']

In [35]:
query = create_contextual_query(lemmas,context_length=50)

In [36]:
print (query)

([lemma="lose"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="lose"])


In [37]:
fragments = {}

In [38]:
fragments['main_node'] = main_node

In [39]:
fragments['mid_node'] = domain_term
fragments['fragments']= []

In [40]:
fragment_1 = {}
fragment_1['original_sentence'] = 'By the time the Nazis took us into the concentration camp I was so numb inside, I was so dead, I was not able to feel anything.'
fragment_1['label']='(..) I was so numb inside, I was so dead (..)'
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22By%22%5D+%5B%22the%22%5D+%5B%22time%22%5D+%5B%22the%22%5D+%5B%22Nazis%22%5D+%5B%22took%22%5D+%5B%22us%22%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%22concentration%22%5D+%5B%22camp%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%22inside%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22so%22%5D+%5B%22dead%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22not%22%5D+%5B%22able%22%5D+%5B%22to%22%5D+%5B%22feel%22%5D+%5B%22anything%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'By the time the Nazis took us into the concentration camp I was so numb inside , I was so dead , I was not able to feel anything . ', 'right': '', 'complete_match': 'By the time the Nazis took us into the concentration camp I was so numb inside , I was so dead , I was not able to feel anything . ', 'testimony_id': 'irn504792', 'shelfmark': ['USHMM RG-50.030*0296'], 'token_s

In [41]:
fragment_2 = {}
fragment_2['original_sentence'] = 'You were so numb that everything that you saw around you, that you, your feelings were dead.'
fragment_2['label']='You were so numb that everything that you saw around you, that you, your feelings were dead.'
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22were%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%22that%22%5D+%5B%22everything%22%5D+%5B%22that%22%5D+%5B%22you%22%5D+%5B%22saw%22%5D+%5B%22around%22%5D+%5B%22you%22%5D+%5B%5D+%5B%22that%22%5D+%5B%22you%22%5D+%5B%5D+%5B%22your%22%5D+%5B%22feelings%22%5D+%5B%22were%22%5D+%5B%22dead%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'You were so numb that everything that you saw around you , that you , your feelings were dead . ', 'right': '', 'complete_match': 'You were so numb that everything that you saw around you , that you , your feelings were dead . ', 'testimony_id': 'irn505576', 'shelfmark': ['USHMM RG-50.042*0023'], 'token_start': 8257, 'token_end': 8277}


In [42]:
fragment_3 = {}
fragment_3['original_sentence'] = 'Uh, we had, we were moving sometimes uh, fifty or sixty miles a day and working day and night and uh, we were just uh, uh, we were just dead on our feet, and uh, one of the problems that we had in relation to this type of recollection is that uh, we were numb.'
fragment_3['label']='we were just dead on our feet, and uh, one of the problems that we had in relation to this type of recollection is that uh, we were numb.'
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Uh%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22had%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22moving%22%5D+%5B%22sometimes%22%5D+%5B%22uh%22%5D+%5B%5D+%5B%22fifty%22%5D+%5B%22or%22%5D+%5B%22sixty%22%5D+%5B%22miles%22%5D+%5B%22a%22%5D+%5B%22day%22%5D+%5B%22and%22%5D+%5B%22working%22%5D+%5B%22day%22%5D+%5B%22and%22%5D+%5B%22night%22%5D+%5B%22and%22%5D+%5B%22uh%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22just%22%5D+%5B%22uh%22%5D+%5B%5D+%5B%22uh%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22just%22%5D+%5B%22dead%22%5D+%5B%22on%22%5D+%5B%22our%22%5D+%5B%22feet%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22uh%22%5D+%5B%5D+%5B%22one%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22problems%22%5D+%5B%22that%22%5D+%5B%22we%22%5D+%5B%22had%22%5D+%5B%22in%22%5D+%5B%22relation%22%5D+%5B%22to%22%5D+%5B%22this%22%5D+%5B%22type%22%5D+%5B%22of%22%5D+%5B%22recollection%22%5D+%5B%22is%22%5D+%5B%22that%22%5D+%5B%22uh%22%5D+%5B%5D+%5B%22we%22%5D+%5

In [43]:
fragment_4 = {}
fragment_4['original_sentence'] = 'Your heart, everything, your whole body start getting already like numb and dead.'
fragment_4['label']="Your heart, everything, your whole body start getting already like numb and dead."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Your%22%5D+%5B%22heart%22%5D+%5B%5D+%5B%22everything%22%5D+%5B%5D+%5B%22your%22%5D+%5B%22whole%22%5D+%5B%22body%22%5D+%5B%22start%22%5D+%5B%22getting%22%5D+%5B%22already%22%5D+%5B%22like%22%5D+%5B%22numb%22%5D+%5B%22and%22%5D+%5B%22dead%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Your heart , everything , your whole body start getting already like numb and dead . ', 'right': '', 'complete_match': 'Your heart , everything , your whole body start getting already like numb and dead . ', 'testimony_id': 'irn510703', 'shelfmark': ['USHMM RG-50.156*0049'], 'token_start': 1038, 'token_end': 1054}


In [44]:
add_testimonial_fragments(fragments)

mid node exists cannot be added


### 4.  Lose

In [45]:
lemmas =['lose','numb']

In [46]:
query = create_contextual_query(lemmas)

In [47]:
print (query)

([lemma="lose"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="lose"])


In [48]:
domain_term = 'lose'

In [49]:
fragments = {}

In [50]:
fragments['main_node'] = main_node

In [51]:
fragments['mid_node'] = domain_term
fragments['fragments']= []

In [52]:
fragment_1 = {}
fragment_1['original_sentence'] = "I saw parents who lost their children. I saw husbands who lost their wives and vise versa, and they were numbed, actually, to-- they didn't complain."
fragment_1['label']="I saw parents who lost their children. I saw husbands who lost their wives and vise versa, and they were numbed (..)"
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22saw%22%5D+%5B%22parents%22%5D+%5B%22who%22%5D+%5B%22lost%22%5D+%5B%22their%22%5D+%5B%22children%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22saw%22%5D+%5B%22husbands%22%5D+%5B%22who%22%5D+%5B%22lost%22%5D+%5B%22their%22%5D+%5B%22wives%22%5D+%5B%22and%22%5D+%5B%22vise%22%5D+%5B%22versa%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22numbed%22%5D+%5B%5D+%5B%22actually%22%5D+%5B%5D+%5B%22to%22%5D+%5B%5D%7B0%2C3%7D+%5B%22they%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22complain%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "I saw parents who lost their children . I saw husbands who lost their wives and vise versa , and they were numbed , actually , to -- they did n't complain . ", 'right': '', 'complete_match': "I saw parents who lost their children . I saw husbands who lost their wives and vise versa , and they were numbed , actually , to -- they did 

In [53]:
fragment_2 = {}
fragment_2['original_sentence'] = "2:3:18 The most tragic kind of family is the numb family who are survivors who usually lost both their parents and their children."
fragment_2['label']="(..) the numb family who are survivors who usually lost both their parents and their children."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)




http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%222%3A3%3A18%22%5D+%5B%22The%22%5D+%5B%22most%22%5D+%5B%22tragic%22%5D+%5B%22kind%22%5D+%5B%22of%22%5D+%5B%22family%22%5D+%5B%22is%22%5D+%5B%22the%22%5D+%5B%22numb%22%5D+%5B%22family%22%5D+%5B%22who%22%5D+%5B%22are%22%5D+%5B%22survivors%22%5D+%5B%22who%22%5D+%5B%22usually%22%5D+%5B%22lost%22%5D+%5B%22both%22%5D+%5B%22their%22%5D+%5B%22parents%22%5D+%5B%22and%22%5D+%5B%22their%22%5D+%5B%22children%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': '2:3:18 The most tragic kind of family is the numb family who are survivors who usually lost both their parents and their children . ', 'right': '', 'complete_match': '2:3:18 The most tragic kind of family is the numb family who are survivors who usually lost both their parents and their children . ', 'testimony_id': 'irn514120', 'shelfmark': ['USHMM RG-50.243*0007'], 'token_start': 1326, 'token_end': 1350}


In [54]:
fragment_3 = {}
fragment_3['original_sentence'] = "I still was despising...but I had become somehow numb. I knew I couldn't change things...all these people I have lost, all these victims I had seen, all these corpses I had seen lying around, and by the way I lost also then within a short time, two of my close friends be...because they they could...one was a man who was highly idealistic and he couldn't take the misery and the the this this this this height of inhumanity, so he he uh said enough is enough."
fragment_3['label']="I had become somehow numb. I knew I couldn't change things...all these people I have lost"
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22still%22%5D+%5B%22was%22%5D+%5B%22despising%22%5D+%5B%5D%7B0%2C50%7D+%5B%22but%22%5D+%5B%22I%22%5D+%5B%22had%22%5D+%5B%22become%22%5D+%5B%22somehow%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22knew%22%5D+%5B%22I%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22change%22%5D+%5B%22things%22%5D+%5B%5D%7B0%2C50%7D+%5B%22all%22%5D+%5B%22these%22%5D+%5B%22people%22%5D+%5B%22I%22%5D+%5B%22have%22%5D+%5B%22lost%22%5D+%5B%5D+%5B%22all%22%5D+%5B%22these%22%5D+%5B%22victims%22%5D+%5B%22I%22%5D+%5B%22had%22%5D+%5B%22seen%22%5D+%5B%5D+%5B%22all%22%5D+%5B%22these%22%5D+%5B%22corpses%22%5D+%5B%22I%22%5D+%5B%22had%22%5D+%5B%22seen%22%5D+%5B%22lying%22%5D+%5B%22around%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22by%22%5D+%5B%22the%22%5D+%5B%22way%22%5D+%5B%22I%22%5D+%5B%22lost%22%5D+%5B%22also%22%5D+%5B%22then%22%5D+%5B%22within%22%5D+%5B%22a%22%5D+%5B%22short%22%5D+%5B%22time%22%5D+%5B%5D+%5B%22two%22%5D+%5B%22of%22%5D+%5B%22my%22%5D+%5B%22c

In [55]:
fragment_4 = {}
fragment_4['original_sentence'] = "MR: I was so numb that I don't even know. I was all numb. I lost everybody."
fragment_4['label']="I was all numb. I lost everybody."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22MR%22%5D+%5B%22%3A%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%22that%22%5D+%5B%22I%22%5D+%5B%22do%22%5D+%5B%5D+%5B%22even%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22all%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22lost%22%5D+%5B%22everybody%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "MR : I was so numb that I do n't even know . I was all numb . I lost everybody . ", 'right': '', 'complete_match': "MR : I was so numb that I do n't even know . I was all numb . I lost everybody . ", 'testimony_id': 'usc_shoah_198', 'shelfmark': ['USC 198'], 'token_start': 8130, 'token_end': 8152}


In [56]:
add_testimonial_fragments(fragments)

### 5.  No feeling

In [57]:
lemmas =['feeling','numb']

In [58]:
query = create_contextual_query(lemmas)

In [59]:
print (query)

([lemma="feeling"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="feeling"])


In [60]:
domain_term = 'no feeling'

In [61]:
fragments = {}

In [62]:
fragments['main_node'] = main_node

In [63]:
fragments['mid_node'] = domain_term
fragments['fragments']= []

In [64]:
fragment_1 = {}
fragment_1['original_sentence'] = "We were numb. The five years there were no feelings."
fragment_1['label']="We were numb. The five years there were no feelings."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22were%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%22The%22%5D+%5B%22five%22%5D+%5B%22years%22%5D+%5B%22there%22%5D+%5B%22were%22%5D+%5B%22no%22%5D+%5B%22feelings%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We were numb . The five years there were no feelings . ', 'right': '', 'complete_match': 'We were numb . The five years there were no feelings . ', 'testimony_id': 'HVT-81', 'shelfmark': ['Fortunoff HVT-81'], 'token_start': 8935, 'token_end': 8947}


In [65]:
fragment_2 = {}
fragment_2['original_sentence'] = "They had no feeling to even think how I would feel. And so I'm just like a machine, numb machine, going through the mechanical motions of something that I was forced into doing."
fragment_2['label']="They had no feeling to even think how I would feel. And so I'm just like a machine, numb machine (..)"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22had%22%5D+%5B%22no%22%5D+%5B%22feeling%22%5D+%5B%22to%22%5D+%5B%22even%22%5D+%5B%22think%22%5D+%5B%22how%22%5D+%5B%22I%22%5D+%5B%22would%22%5D+%5B%22feel%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22so%22%5D+%5B%22I%22%5D+%5B%22%27m%22%5D+%5B%22just%22%5D+%5B%22like%22%5D+%5B%22a%22%5D+%5B%22machine%22%5D+%5B%5D+%5B%22numb%22%5D+%5B%22machine%22%5D+%5B%5D+%5B%22going%22%5D+%5B%22through%22%5D+%5B%22the%22%5D+%5B%22mechanical%22%5D+%5B%22motions%22%5D+%5B%22of%22%5D+%5B%22something%22%5D+%5B%22that%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22forced%22%5D+%5B%22into%22%5D+%5B%22doing%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "They had no feeling to even think how I would feel . And so I 'm just like a machine , numb machine , going through the mechanical motions of something that I was forced into doing . ", 'right': '', 'complete_match': "They had no fee

In [66]:
fragment_3 = {}
fragment_3['original_sentence'] = "But you become numb, you know. You...you're not really sensitive to feelings any more."
fragment_3['label']="But you become numb, you know. You...you're not really sensitive to feelings any more."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22But%22%5D+%5B%22you%22%5D+%5B%22become%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22You%22%5D+%5B%5D%7B0%2C50%7D+%5B%22you%22%5D+%5B%5D+%5B%22not%22%5D+%5B%22really%22%5D+%5B%22sensitive%22%5D+%5B%22to%22%5D+%5B%22feelings%22%5D+%5B%22any%22%5D+%5B%22more%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "But you become numb , you know . You ... you 're not really sensitive to feelings any more . ", 'right': '', 'complete_match': "But you become numb , you know . You ... you 're not really sensitive to feelings any more . ", 'testimony_id': 'irn504449', 'shelfmark': ['USHMM RG-50.030*0020'], 'token_start': 15269, 'token_end': 15289}


In [67]:
fragment_4 = {}
fragment_4['original_sentence'] = "The were no feelings. You were completely numb."
fragment_4['label']= "The were no feelings. You were completely numb."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22The%22%5D+%5B%22were%22%5D+%5B%22no%22%5D+%5B%22feelings%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22were%22%5D+%5B%22completely%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'The were no feelings . You were completely numb . ', 'right': '', 'complete_match': 'The were no feelings . You were completely numb . ', 'testimony_id': 'HVT-36', 'shelfmark': ['Fortunoff HVT-36'], 'token_start': 5363, 'token_end': 5373}


In [68]:
fragment_5 = {}
fragment_5['original_sentence'] = "It was all numb and just – couldn’t have any feeling, it felt like just you – you know how your leg – how something goes to sleep when you’re [indecipherable] that’s the way it felt."
fragment_5['label']= "It was all numb and just – couldn’t have any feeling"
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)



http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22It%22%5D+%5B%22was%22%5D+%5B%22all%22%5D+%5B%22numb%22%5D+%5B%22and%22%5D+%5B%22just%22%5D+%5B%5D%7B0%2C3%7D+%5B%22could%22%5D+%5B%5D+%5B%22have%22%5D+%5B%22any%22%5D+%5B%22feeling%22%5D+%5B%5D+%5B%22it%22%5D+%5B%22felt%22%5D+%5B%22like%22%5D+%5B%22just%22%5D+%5B%22you%22%5D+%5B%5D%7B0%2C3%7D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%22how%22%5D+%5B%22your%22%5D+%5B%22leg%22%5D+%5B%5D%7B0%2C3%7D+%5B%22how%22%5D+%5B%22something%22%5D+%5B%22goes%22%5D+%5B%22to%22%5D+%5B%22sleep%22%5D+%5B%22when%22%5D+%5B%22you%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22indecipherable%22%5D+%5B%5D%7B0%2C3%7D+%5B%22that%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22way%22%5D+%5B%22it%22%5D+%5B%22felt%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'It was all numb and just – could n’t have any feeling , it felt like just you – you know how your leg – how something goes to sleep when you ’re [ indecipherabl

In [69]:
add_testimonial_fragments(fragments)

### 6.  No cry

In [70]:
lemmas =['cry','numb']

In [71]:
query = create_contextual_query(lemmas)

In [72]:
print (query)

([lemma="cry"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="cry"])


In [73]:
domain_term = 'no cry'
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments']= []

In [74]:
fragment_1 = {}
fragment_1['original_sentence'] = "You never cried. You so...it was so numb...it's...it's very hard to explain."
fragment_1['label']="You never cried. You so...it was so numb...it's...it's very hard to explain."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22never%22%5D+%5B%22cried%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22so%22%5D+%5B%5D%7B0%2C50%7D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%5D%7B0%2C50%7D+%5B%22it%22%5D+%5B%5D+%5B%5D%7B0%2C50%7D+%5B%22it%22%5D+%5B%5D+%5B%22very%22%5D+%5B%22hard%22%5D+%5B%22to%22%5D+%5B%22explain%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "You never cried . You so ... it was so numb ... it 's ... it 's very hard to explain . ", 'right': '', 'complete_match': "You never cried . You so ... it was so numb ... it 's ... it 's very hard to explain . ", 'testimony_id': 'irn504680', 'shelfmark': ['USHMM RG-50.030*0184'], 'token_start': 12235, 'token_end': 12257}


In [75]:
fragment_2 = {}
fragment_2['original_sentence'] = "That was--we were numb. We couldn't--it wasn't a question--we weren't crying but we were--we were like num--just pieces of would."
fragment_2['label']="(..)we were numb. We couldn't--it wasn't a question--we weren't crying (..)"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22That%22%5D+%5B%22was%22%5D+%5B%5D%7B0%2C3%7D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22could%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%5D+%5B%22a%22%5D+%5B%22question%22%5D+%5B%5D%7B0%2C3%7D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%5D+%5B%22crying%22%5D+%5B%22but%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%5D%7B0%2C3%7D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22like%22%5D+%5B%22num%22%5D+%5B%5D%7B0%2C3%7D+%5B%22just%22%5D+%5B%22pieces%22%5D+%5B%22of%22%5D+%5B%22would%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "That was -- we were numb . We could n't -- it was n't a question -- we were n't crying but we were -- we were like num -- just pieces of would . ", 'right': '', 'complete_match': "That was -- we were numb . We could n't -- it was n't a question -- we were n't crying but we were -- we were like num -- just 

In [76]:
fragment_3 = {}
fragment_3['original_sentence'] = "But I-- at that time, I was like-- I felt like I, I was-- I couldn't even cry anymore. Just couldn't. I mean, there just-- I was numb."
fragment_3['label']="I couldn't even cry anymore. Just couldn't. I mean, there just-- I was numb."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22But%22%5D+%5B%22I%22%5D+%5B%5D%7B0%2C3%7D+%5B%22at%22%5D+%5B%22that%22%5D+%5B%22time%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22like%22%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22felt%22%5D+%5B%22like%22%5D+%5B%22I%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22even%22%5D+%5B%22cry%22%5D+%5B%22anymore%22%5D+%5B%5D+%5B%22Just%22%5D+%5B%22could%22%5D+%5B%5D+%5B%5D+%5B%22I%22%5D+%5B%22mean%22%5D+%5B%5D+%5B%22there%22%5D+%5B%22just%22%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "But I -- at that time , I was like -- I felt like I , I was -- I could n't even cry anymore . Just could n't . I mean , there just -- I was numb . ", 'right': '', 'complete_match': "But I -- at that time , I was like -- I felt like I , I was -- I could n't even cry a

In [77]:
fragment_4 = {}
fragment_4['original_sentence'] = "And after about four hours-- and I probably was numb; I couldn't cry or say [?"
fragment_4['label']="I probably was numb; I couldn't cry"
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22after%22%5D+%5B%22about%22%5D+%5B%22four%22%5D+%5B%22hours%22%5D+%5B%5D%7B0%2C3%7D+%5B%22and%22%5D+%5B%22I%22%5D+%5B%22probably%22%5D+%5B%22was%22%5D+%5B%22numb%22%5D+%5B%22%3B%22%5D+%5B%22I%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22cry%22%5D+%5B%22or%22%5D+%5B%22say%22%5D+%5B%5D%7B0%2C3%7D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And after about four hours -- and I probably was numb ; I could n't cry or say [ ", 'right': '', 'complete_match': "And after about four hours -- and I probably was numb ; I could n't cry or say [ ", 'testimony_id': 'usc_shoah_6845', 'shelfmark': ['USC 6845'], 'token_start': 6497, 'token_end': 6516}


In [78]:
fragment_5 = {}
fragment_5['original_sentence'] = "I was numb. I didn’t express any emotions or anything like that, I didn’t cry."
fragment_5['label']="I was numb. I didn’t express any emotions or anything like that, I didn’t cry."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22was%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22express%22%5D+%5B%22any%22%5D+%5B%22emotions%22%5D+%5B%22or%22%5D+%5B%22anything%22%5D+%5B%22like%22%5D+%5B%22that%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22cry%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I was numb . I did n’t express any emotions or anything like that , I did n’t cry . ', 'right': '', 'complete_match': 'I was numb . I did n’t express any emotions or anything like that , I did n’t cry . ', 'testimony_id': 'irn507294', 'shelfmark': ['USHMM RG-50.030*0405'], 'token_start': 29197, 'token_end': 29217}


In [79]:
add_testimonial_fragments(fragments)